In [1]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [3]:
strategy = tf.distribute.TPUStrategy(resolver)

In [4]:
!pip install hickle
import hickle as hkl
import os
import gc
import matplotlib.pyplot as plt
from keras import layers,Input,Sequential
from keras.layers import Dense,Flatten,Reshape,Conv2DTranspose,Conv2D,MaxPooling2D,Reshape,Resizing,Dropout,UpSampling2D
from tensorflow.keras import datasets, layers, models, losses, Model
from random import randint
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [5]:
with strategy.scope():
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    reconstructed_front = tf.keras.models.load_model('/gdrive/My Drive/Models_Static/Frontal', options=load_locally)
    reconstructed_left= tf.keras.models.load_model('/gdrive/My Drive/Models_Static/Left', options=load_locally)
    reconstructed_left45 = tf.keras.models.load_model('/gdrive/My Drive/Models_Static/Left45', options=load_locally)
    reconstructed_right = tf.keras.models.load_model('/gdrive/My Drive/Models_Static/Right', options=load_locally)
    reconstructed_right45 = tf.keras.models.load_model('/gdrive/My Drive/Models_Static/Right45', options=load_locally)

In [6]:
load_hkl_frontal = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets/Static/Frontal.hkl")
X_train_frontal=load_hkl_frontal['xtrain'] / 255.
X_test_frontal=load_hkl_frontal['xtest'] / 255.
y_train_frontal=load_hkl_frontal['ytrain']
y_test_frontal=load_hkl_frontal['ytest']
del load_hkl_frontal
gc.collect()

443

In [7]:
load_hkl_left = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets/Static/Left.hkl")
X_train_left=load_hkl_left['xtrain'] / 255.
X_test_left=load_hkl_left['xtest'] / 255.
del load_hkl_left
gc.collect()

0

In [8]:
load_hkl_left45 = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets/Static/Left45.hkl")
X_train_left45=load_hkl_left45['xtrain'] / 255.
X_test_left45=load_hkl_left45['xtest'] / 255.
del load_hkl_left45
gc.collect()

0

In [9]:
load_hkl_right = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets/Static/Right.hkl")
X_train_right=load_hkl_right['xtrain'] / 255.
X_test_right=load_hkl_right['xtest'] / 255.
del load_hkl_right
gc.collect()

0

In [10]:
load_hkl_right45 = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets/Static/Right45.hkl")
X_train_right45=load_hkl_right45['xtrain'] / 255.
X_test_right45=load_hkl_right45['xtest'] / 255.
del load_hkl_right45
gc.collect()

0

In [11]:
xm_train_frontal=reconstructed_front.encoder(X_train_frontal)
xm_test_frontal=reconstructed_front.encoder(X_test_frontal)
del X_train_frontal
del X_test_frontal
gc.collect()

0

In [12]:
xm_train_left=reconstructed_left.encoder(X_train_left)
xm_test_left=reconstructed_left.encoder(X_test_left)
del X_train_left
del X_test_left
gc.collect()

0

In [13]:
xm_train_left45=reconstructed_left45.encoder(X_train_left45)
xm_test_left45=reconstructed_left45.encoder(X_test_left45)
del X_train_left45
del X_test_left45
gc.collect()

0

In [14]:
xm_train_right=reconstructed_right.encoder(X_train_right)
xm_test_right=reconstructed_right.encoder(X_test_right)
del X_train_right
del X_test_right
gc.collect()

0

In [15]:
xm_train_right45=reconstructed_right45.encoder(X_train_right45)
xm_test_right45=reconstructed_right45.encoder(X_test_right45)
del X_train_right45
del X_test_right45
gc.collect()

0

In [16]:
xm_train_frontal=tf.reshape(xm_train_frontal,[xm_train_frontal.shape[0],-1])
xm_test_frontal=tf.reshape(xm_test_frontal,[xm_test_frontal.shape[0],-1])

xm_train_left=tf.reshape(xm_train_left,[xm_train_left.shape[0],-1])
xm_test_left=tf.reshape(xm_test_left,[xm_test_left.shape[0],-1])

xm_train_left45=tf.reshape(xm_train_left45,[xm_train_left45.shape[0],-1])
xm_test_left45=tf.reshape(xm_test_left45,[xm_test_left45.shape[0],-1])

xm_train_right=tf.reshape(xm_train_right,[xm_train_right.shape[0],-1])
xm_test_right=tf.reshape(xm_test_right,[xm_test_right.shape[0],-1])

xm_train_right45=tf.reshape(xm_train_right45,[xm_train_right45.shape[0],-1])
xm_test_right45=tf.reshape(xm_test_right45,[xm_test_right45.shape[0],-1])


In [17]:
xm_train_frontal.shape

TensorShape([194, 614400])

In [18]:
xm_test_frontal.shape[0]

83

In [46]:
  xm_train_vector=np.empty((0,xm_train_frontal.shape[1]),dtype='float32')
  print(xm_train_vector.shape)
  y_train_vector=y_train_frontal
  xm_test_vector=np.empty((0,xm_test_frontal.shape[1]),dtype='float32')
  y_test_vector=y_test_frontal

  for i in range(xm_train_frontal.shape[0]):

      xm_train_vector_1=np.empty((0,xm_train_frontal.shape[1]),dtype='float32')


      xm_train_vector_1=np.concatenate((xm_train_vector_1,np.reshape(xm_train_frontal[i,:],(1,-1))),axis=0)
      xm_train_vector_1=np.concatenate((xm_train_vector_1,np.reshape(xm_train_left[i,:],(1,-1))),axis=0)
      xm_train_vector_1=np.concatenate((xm_train_vector_1,np.reshape(xm_train_left45[i,:],(1,-1))),axis=0)
      xm_train_vector_1=np.concatenate((xm_train_vector_1,np.reshape(xm_train_right[i,:],(1,-1))),axis=0)
      xm_train_vector_1=np.concatenate((xm_train_vector_1,np.reshape(xm_train_right45[i,:],(1,-1))),axis=0)
      print(xm_train_vector_1.shape)
      xm_train_vector=np.concatenate((xm_train_vector,np.reshape(xm_train_vector_1.max(axis=0),(1,-1))),axis=0)

  for j in range(xm_test_frontal.shape[0]):

      xm_test_vector_1=np.empty((0,xm_test_frontal.shape[1]),dtype='float32')

      xm_test_vector_1=np.concatenate((xm_test_vector_1,np.reshape(xm_test_frontal[j,:],(1,-1))),axis=0)
      xm_test_vector_1=np.concatenate((xm_test_vector_1,np.reshape(xm_test_left[j,:],(1,-1))),axis=0)
      xm_test_vector_1=np.concatenate((xm_test_vector_1,np.reshape(xm_test_left45[j,:],(1,-1))),axis=0)
      xm_test_vector_1=np.concatenate((xm_test_vector_1,np.reshape(xm_test_right[j,:],(1,-1))),axis=0)
      xm_test_vector_1=np.concatenate((xm_test_vector_1,np.reshape(xm_test_right45[j,:],(1,-1))),axis=0)

      xm_test_vector=np.concatenate((xm_test_vector,np.reshape(xm_test_vector_1.max(axis=0),(1,-1))),axis=0)


  del xm_train_vector_1
  del xm_test_vector_1
  del xm_train_frontal
  del xm_train_left
  del xm_train_left45
  del xm_train_right
  del xm_train_right45
  print(xm_train_vector.shape)
  gc.collect()

(0, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 614400)
(5, 

81

In [47]:
xm_train_vector[0,1:]

array([0.16161886, 0.3397606 , 0.12291701, ..., 0.05406318, 0.1752338 ,
       0.00341835], dtype=float32)

In [39]:
xm_train_vector_1.max(axis=0).shape

(614400,)

In [48]:
xm_train_frontal.shape

NameError: name 'xm_train_frontal' is not defined

## **Machine Learning**

In [56]:
from sklearn.metrics import classification_report

In [57]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

xm_train_vector=scaler.fit_transform(xm_train_vector)
xm_test_vector=scaler.transform(xm_test_vector)
gc.collect()

54

In [58]:
from sklearn.svm import SVC

svc=SVC(kernel="sigmoid")
gc.collect()

0

In [52]:
svc.fit(xm_train_vector, y_train_vector)
#del xm_train_vector
#del y_train_vector
gc.collect()

0

In [53]:
y_pred_vector=svc.predict(xm_test_vector)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm=confusion_matrix(y_test_vector, y_pred_vector)
print(cm)
print(classification_report(y_test_vector, y_pred_vector))

[[51  2]
 [ 8 22]]
              precision    recall  f1-score   support

         0.0       0.86      0.96      0.91        53
         1.0       0.92      0.73      0.81        30

    accuracy                           0.88        83
   macro avg       0.89      0.85      0.86        83
weighted avg       0.88      0.88      0.88        83



In [54]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(n_estimators=30, criterion="entropy", random_state=0)

rfc.fit(xm_train_vector, y_train_vector)
y_pred_vector=rfc.predict(xm_test_vector)

cm=confusion_matrix(y_test_vector, y_pred_vector)
print(cm)

[[50  3]
 [ 8 22]]


In [55]:
print(classification_report(y_test_vector, y_pred_vector))

              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90        53
         1.0       0.88      0.73      0.80        30

    accuracy                           0.87        83
   macro avg       0.87      0.84      0.85        83
weighted avg       0.87      0.87      0.86        83

